In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import numpy as np

Pre-Processing

In [13]:
#Resizing 512x512 and Hair-Removal
#find . -name ".DS_Store" -delete

input_folder = 'skin-lesions/train/melanoma/'

# Specify the output folder to save resized images
output_folder = 'pre-proccessed/train/melanoma/'

# Ensure the output folder exists, if not, create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop through all files in the input folder
for filename in os.listdir(input_folder):
    # Read the image
    img_path = os.path.join(input_folder, filename)
    original_image = cv2.imread(img_path)
    
    # Check if the file is an image
    if original_image is not None:
        # Resize the image to 512x512
        resized_image = cv2.resize(original_image, (224, 224))
        grayScale = cv2.cvtColor( resized_image, cv2.COLOR_RGB2GRAY )
        kernel = cv2.getStructuringElement(1,(17,17))
        blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
        bhg= cv2.GaussianBlur(blackhat,(3,3),cv2.BORDER_DEFAULT)
        ret,thresh2 = cv2.threshold(bhg,10,255,cv2.THRESH_BINARY)
        dst = cv2.inpaint(resized_image,thresh2,1,cv2.INPAINT_TELEA)

        
        # Define the output path for the resized image
        output_path = os.path.join(output_folder, filename)
        
        # Write the resized image to disk
        cv2.imwrite(output_path, dst)

Model Training

In [4]:
from tensorflow.keras.utils import to_categorical
from PIL import Image

data_folder = 'pre-proccessed/train/'

# List to store train images and labels
train_images = []
train_labels = []

# Dictionary to map class names to numerical labels
class_to_label = {'melanoma': 0, 'nevus': 1, 'seborrheic_keratosis': 2}  # Update class names accordingly

# Iterate through the subfolders in the train data folder
for class_name in os.listdir(data_folder):
    class_folder = os.path.join(data_folder, class_name)
    
    # Check if it's a directory (subfolder representing a class)
    if os.path.isdir(class_folder):
        # Assign a numerical label to the class
        class_label = class_to_label[class_name]
        
        # Iterate through the files in the class folder
        for filename in os.listdir(class_folder):
            # Load the image
            image_path = os.path.join(class_folder, filename)
            image = Image.open(image_path)
            
            # Convert image to numpy array and normalize pixel values
            image_array = np.array(image) / 255.0
            
            # Append the image and corresponding label to the train data lists
            train_images.append(image_array)
            train_labels.append(class_label)

# Convert the lists to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Convert labels to one-hot encoding
train_labels_one_hot = to_categorical(train_labels, num_classes=3)

In [13]:
data_folder = 'pre-proccessed/valid/'

# List to store train images and labels
val_images = []
val_labels = []

# Dictionary to map class names to numerical labels
class_to_label = {'melanoma': 0, 'nevus': 1, 'seborrheic_keratosis': 2}  # Update class names accordingly

# Iterate through the subfolders in the train data folder
for class_name in os.listdir(data_folder):
    class_folder = os.path.join(data_folder, class_name)
    
    # Check if it's a directory (subfolder representing a class)
    if os.path.isdir(class_folder):
        # Assign a numerical label to the class
        class_label = class_to_label[class_name]
        
        # Iterate through the files in the class folder
        for filename in os.listdir(class_folder):
            # Load the image
            image_path = os.path.join(class_folder, filename)
            image = Image.open(image_path)
            
            # Convert image to numpy array and normalize pixel values
            image_array = np.array(image) / 255.0
            
            # Append the image and corresponding label to the train data lists
            val_images.append(image_array)
            val_labels.append(class_label)

# Convert the lists to numpy arrays
val_images = np.array(train_images)
val_labels = np.array(train_labels)

# Convert labels to one-hot encoding
val_labels_one_hot = to_categorical(val_labels, num_classes=3)

In [19]:
#ResNet50
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for your specific classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)  # 3 classes, so the output layer has 3 units

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pre-trained ResNet-50 model
for layer in base_model.layers:
    layer.trainable = False
# Compile the model
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate= 0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_8[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [20]:
# Train the model with your pre-processed and augmented data
model.fit(train_images, train_labels_one_hot, epochs=10, batch_size=32, verbose = 1, validation_data = (val_images, val_labels_one_hot))

Epoch 1/10


2023-11-08 13:33:53.037219: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - ETA: 0s - loss: 0.9824 - accuracy: 0.6625

2023-11-08 13:34:26.139476: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 58s 793ms/step - loss: 0.9824 - accuracy: 0.6625 - val_loss: 0.8599 - val_accuracy: 0.6860
Epoch 2/10
63/63 [==============================] - 23s 358ms/step - loss: 0.9111 - accuracy: 0.6770 - val_loss: 0.8573 - val_accuracy: 0.6860
Epoch 3/10
63/63 [==============================] - 22s 358ms/step - loss: 0.8603 - accuracy: 0.6865 - val_loss: 0.8831 - val_accuracy: 0.6880
Epoch 4/10
63/63 [==============================] - 23s 368ms/step - loss: 0.8759 - accuracy: 0.6785 - val_loss: 0.9150 - val_accuracy: 0.6860
Epoch 5/10
63/63 [==============================] - 22s 354ms/step - loss: 0.8592 - accuracy: 0.6860 - val_loss: 0.8299 - val_accuracy: 0.6860
Epoch 6/10
63/63 [==============================] - 22s 358ms/step - loss: 0.8519 - accuracy: 0.6860 - val_loss: 0.8597 - val_accuracy: 0.6885
Epoch 7/10
63/63 [==============================] - 22s 352ms/step - loss: 0.8472 - accuracy: 0.6860 - val_loss: 0.8792 - val_accuracy: 0.6890
Epoch 8/10

In [23]:
#MobileNet
from tensorflow.keras.applications import MobileNetV2
Mobile_net_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for your specific classification task
x = Mobile_net_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)  # 3 classes, so the output layer has 3 units

# Create the final model
model_mobile_net = Model(inputs=Mobile_net_model.input, outputs=predictions)

# Freeze the layers of the pre-trained ResNet-50 model
for layer in Mobile_net_model.layers:
    layer.trainable = False
# Compile the model
model_mobile_net.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate= 0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model_mobile_net.summary()


9406464/9406464 [==============================] - 1s 0us/step
Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_10[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           

In [24]:
model_mobile_net.fit(train_images, train_labels_one_hot, epochs=10, batch_size=32, verbose = 1, validation_data = (val_images, val_labels_one_hot))

Epoch 1/10


2023-11-08 13:42:21.273338: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - ETA: 0s - loss: 0.7991 - accuracy: 0.6645

2023-11-08 13:42:55.080803: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 51s 674ms/step - loss: 0.7991 - accuracy: 0.6645 - val_loss: 0.6609 - val_accuracy: 0.7195
Epoch 2/10
63/63 [==============================] - 10s 159ms/step - loss: 0.6421 - accuracy: 0.7265 - val_loss: 0.6107 - val_accuracy: 0.7350
Epoch 3/10
63/63 [==============================] - 10s 156ms/step - loss: 0.6070 - accuracy: 0.7340 - val_loss: 0.5664 - val_accuracy: 0.7655
Epoch 4/10
63/63 [==============================] - 10s 155ms/step - loss: 0.5794 - accuracy: 0.7540 - val_loss: 0.5399 - val_accuracy: 0.7755
Epoch 5/10
63/63 [==============================] - 10s 154ms/step - loss: 0.5499 - accuracy: 0.7705 - val_loss: 0.5415 - val_accuracy: 0.7615
Epoch 6/10
63/63 [==============================] - 10s 155ms/step - loss: 0.5376 - accuracy: 0.7820 - val_loss: 0.5038 - val_accuracy: 0.7880
Epoch 7/10
63/63 [==============================] - 9s 150ms/step - loss: 0.5117 - accuracy: 0.7890 - val_loss: 0.4837 - val_accuracy: 0.8020
Epoch 8/10


In [ ]:
#Xception

Initilialization of Voting Algorithm

In [ ]:
#SoftVoting